# Объектно-ориентированное программирование. Часть 3

## **S.O.L.I.D** принципы

> **SOLID** — это мнемоническая аббревиатура для набора принципов проектирования, созданных для разработки программного обеспечения при помощи объектно-ориентированных языков. 

Принципы **SOLID** направленны на содействие разработки более простого, надежного и обновляемого кода. 

Каждая буква в аббревиатуре **SOLID** соответствует одному принципу разработки.

При правильной реализации это делает ваш код более расширяемым, логичным и легким для чтения.


> Подходы к разработке __SOLID__ — это абстрактные сущности, которые не привязаны к конкретному языку программирования. Они содержат такие рекомендации для разработчиков:

- __S (SRP) — Single-responsibility principle (единственная ответственность)__. Любой класс должен иметь одну зону ответственности.
- __O — Open–closed principle (открытость и закрытость)__. Классы можно расширять, но желательно напрямую их не модифицировать. Другими словами, код, который уже создан, не должен подвергаться правкам. Разработчик имеет право только добавить что-то или исправить обнаруженные ошибки.
- __L (LSP) — Liskov substitution principle (правило подстановки Барбары Лисков)__. Этот принцип самый трудный для понимания и немного абстрактный. Речь идет о логичности наследования; о том, что класс-предок можно поменять на дочерний, не ломая логику работы программы.
- __I (ISP) — Interface segregation principle (разделение интерфейса)__. Суть этого принципа в преимуществе интерфейса, специально предназначенного для клиентов по сравнению с единым интерфейсом общего назначения для всех.
- __D (DIP) — Dependency inversion principle (правило инверсии зависимостей)__. Более высокоуровневые модули не должны зависеть от более низкоуровневых, а в идеале они должны зависеть от некоторых абстракций. Детали не должны оказывать влияние на абстракции, а скорее абстракции должны влиять на детали.

'<img src="img/solid1.png" width="600">'

### 1. Single Responsibility Principle (Принцип единственной обязанности)


> Принцип единственной обязанности требует того, чтобы один класс выполнял только **одну** работу. 

Таким образом, если у класса есть более одной работы, он становится зависимым. Изменение поведения одной работы класса приводит к изменению в другой.

'<img src="img/solid2.png" width="600">'

In [ ]:
# Below is Given a class which has two responsibilities 
class  User:
    def __init__(self, name: str):
        self.name = name
    
    def get_name(self) -> str:
        pass

    def save(self, user: User):
        pass

Мы имеем класс ```User```, который ответственен за **две** работы — *свойства пользователя* и *управление базой данных*. 

Если в приложении будет изменен функционал управления базой данных для пользователя, тогда классы использующие свойства класса ```User``` тоже придется доработать и перекомпилировать, чтобы компенсировать новые изменения. Это как домино эффект, уроните одну кость, и она уронит все за ней следом.

Мы же просто разделим класс. Мы создадим ещё один класс, который возьмет на себя одну ответственность — управление базой данных пользователя.

In [ ]:
class BaseUser:
    def __init__(self, name: str):
            self.name = name
    
    def get_name(self):
        pass


class UserDB:
    def get_user(self, name) -> User:
        pass

    def save(self, user: User):
        pass


class User:
    def __init__(self, name: str):
        self.b_user = BaseUser(name)
        self.db = UserDB()
    
    def get_name(self):
        return self.b_user.get_name()
    
    def save(self):
        self.db.save() 

Распространённым решением этой проблемы является применение шаблона проектирования **Фасад**.

```User``` класс был бы фасадом для управления базой данных пользователя и управления свойствами пользователя.

In [ ]:
# Плохо: класс выполняет две задачи — управление пользователем и логирование.
class UserManager:
    def __init__(self, user):
        self.user = user

    def change_user_name(self, new_name):
        self.user.name = new_name

    def log_user_change(self, message):
        with open("log.txt", "a") as file:
            file.write(message)

In [5]:
# Хорошо: разделение на два класса, каждый с одной ответственностью.
class UserManager:
    def __init__(self, user):
        self.user = user

    def change_user_name(self, new_name):
        self.user.name = new_name

class Logger:
    def log(self, message):
        with open("log.txt", "a") as file:
            file.write(message)

'<img src="img/solid2_2.jpg" width="600">'

### 2. Open-Closed Principle (Принцип открытости/закрытости)

Программные сущности (классы, модули, функции) должно быть открыты для расширения, но не модификации.

Давайте представим, что у вас есть магазин, и вы даете скидку в 10% для ваших любимых покупателей используя класс ```Discount```. 

Если бы вы решаете удвоить 10-ти процентную скидку для VIP клиентов, вы могли бы изменить класс следующим образом:

In [ ]:
# Плохо: добавление нового типа требует изменения класса.
class Discount:
    def __init__(self, customer_type):
        self.customer_type = customer_type

    def give_discount(self):
        if self.customer_type == "regular":
            return 10
        elif self.customer_type == "premium":
            return 20

Но нет, это нарушает **OCP**. 

> **OCP** запрещает это. Например, если мы хотим дать новую скидку для другого типа покупателей, то это требует добавления новой логики. Чтобы следовать **OCP**, мы добавим новый класс, который будет расширять ```Discount```. И в этом новом классе реализуем требуемую логику:

In [3]:
# Хорошо: использование наследования для расширения.
from abc import ABC, abstractmethod

class Discount(ABC):
    @abstractmethod
    def give_discount(self):
        pass

class RegularDiscount(Discount):
    def give_discount(self):
        return 10

class PremiumDiscount(Discount):
    def give_discount(self):
        return 20

Если вы решите дать скидку супер VIP пользователям, то это будет выглядеть так:

In [ ]:
class SuperVIPDiscount(PremiumDiscount):
    def give_discount(self):
        return super().give_discount() * 2

> **Расширяйте, но не модифицируйте!!!**

### 3. Liskov Substitution Principle (Принцип подстановки Лисков)

Главная идея, стоящая за ```Liskov Substitution Principle``` в том, что для любого класса клиент должен иметь возможность использовать любой подкласс базового класса, не замечая разницы между ними, и следовательно, без каких-либо изменений поведения программы при выполнении. 

> Это означает, что клиент полностью изолирован и не подозревает об изменениях в иерархии классов.

Более формально: Пусть ```q(x)``` является свойством, верным относительно объектов ```x``` некоторого типа ```T```. Тогда ```q(y)``` также должно быть верным для объектов y типа ```S```, где ```S``` является подтипом типа ```T```.

Проще говоря, это значит, что подкласс, дочерний класс должны соответствовать их родительскому классу или супер классу.

**LSP** это основа хорошего объектно-ориентированного проектирования программного обеспечения, потому что он следует одному из базовых принципов ООП — полиморфизму. 

Речь о том, чтобы создавать правильные иерархии, такие, что классы, производные от базового являлись полиморфными для их родителя по отношению к методам их интерфейсов. 

Ещё интересно отметить, как этот принцип относится к примеру предыдущего принципа. 

Если мы пытаемся расширить класс новым несовместимым классом, то все сломается. Взаимодействие с клиентом будет нарушено, и как результат, такое расширение будет невозможно (или, для того чтобы сделать это возможным, нам пришлось бы нарушить другой принцип и модифицировать код клиента, который должен быть закрыт для модификации, такое крайне нежелательно и неприемлемо).

Тщательное обдумывание новых классов в соответствии с **LSP** помогает нам расширять иерархию классов правильно. Также, **LSP** способствует **OCP**.

In [1]:
# Плохо: подкласс нарушает поведение базового класса.
class Bird:
    def fly(self):
        pass

class Ostrich(Bird): # Страус
    def fly(self):
        raise NotImplementedError("Страусы не летают")

In [2]:
# Хорошо: разделение на классы с корректным поведением.
class Bird:
    pass

class FlyingBird(Bird):
    def fly(self):
        pass

class Ostrich(Bird):
    pass

### 4. Interface Segregation Principle (Принцип разделения интерфейсов)

> Создавайте тонкие интерфейсы, которые ориентированы на клиента. Клиенты не должны зависеть от интерфейсов, которые они не используют. Этот принцип устраняет недостатки реализации больших интерфейсов.

'<img src="img/solid3.png" width="600">'

Чтобы полностью проиллюстрировать это, мы возьмем классический пример, потому что он очень показательный и легок для понимания. Классический пример:

In [ ]:
class IShape:
    def draw(self):
        raise NotImplementedError

class Circle(IShape):
    def draw(self):
        pass

class Square(IShape):
    def draw(self):
        pass

class Rectangle(IShape):
    def draw(self):
        pass

Еще один приятный трюк заключается в том, что в нашей бизнес-логике отдельный класс может реализовать несколько интерфейсов, если необходимо. 

Таким образом, мы может предоставить единую реализацию для всех общих методов между интерфейсами. 

Сегрегированные интерфейсы заставляют нас больше думать о нашем коде с точки зрения клиента, что приведет нас к меньшей зависимости и более легкому тестированию. 

Таким образом, мы не только сделали наш код лучше для клиента, но также это облегчило нам понимание, тестирование и реализацию кода для нас самих.

In [ ]:
# Плохо: один интерфейс с избыточными методами.
class Machine:
    def print(self):
        print("print")

    def scan(self):
        print("scan")

    def fax(self):
        print("fax")

In [ ]:
# Хорошо: разделение на несколько интерфейсов.
class Printer:
    def print(self):
        print("print")

class Scanner:
    def scan(self):
        print("scan")

class Fax:
    def fax(self):
        print("fax")

In [ ]:
class Machine(Printer, Scanner, Fax):
    pass

### 5. Dependecy Inversion Principle (Принцип инверсии зависимостей)

> Зависимость должна быть от абстракций, а не от конкретики. 

* Модули верхних уровней не должны зависеть от модулей нижних уровней. 
* Классы и верхних, и нижних уровней должны зависеть от одних и тех же абстракций. 
* Абстракции не должны зависеть от деталей. 
* Детали должны зависеть от абстракций.

Наступает момент в разработке, когда наше приложение в основном состоит из модулей. Когда такое происходит, нам необходимо улучшать код используя внедрение зависимостей. Функционирование компонентов высокого уровня зависит от компонентов низкого уровня. Для создания определенного поведения вы можете использовать наследование или интерфейсы.

In [ ]:
# Плохо: класс напрямую зависит от конкретной реализации.
class LightBulb:
    def turn_on(self):
        print("Лампочка включена")

    def turn_off(self):
        print("Лампочка выключена")

class Switch:
    def __init__(self):
        self.bulb = LightBulb()

    def on(self):
        self.bulb.turn_on()

    def off(self):
        self.bulb.turn_off()

In [ ]:
# Хорошо: зависимость от абстракции.
from abc import ABC, abstractmethod

class Switchable(ABC):
    @abstractmethod
    def turn_on(self):
        pass

    @abstractmethod
    def turn_off(self):
        pass

class LightBulb(Switchable):
    def turn_on(self):
        print("Лампочка включена")

    def turn_off(self):
        print("Лампочка выключена")

class Switch:
    def __init__(self, device: Switchable):
        self.device = device

    def on(self):
        self.device.turn_on()

    def off(self):
        self.device.turn_off()

Или:

In [ ]:
class AuthenticationForUser():
    def __init__(self, connector:Connector):
        self.connection = connector.connect()

    def authenticate(self, credentials):
        pass
    
    def is_authenticated(self):
        pass
    
    def last_login(self):
        pass

class AnonymousAuth(AuthenticationForUser):
    pass

class GithubAuth(AuthenticationForUser):
    def last_login(self):
        pass

class FacebookAuth(AuthenticationForUser):
    pass

class Permissions()
    def __init__(self, auth: AuthenticationForUser)
        self.auth = auth

    def has_permissions():
        pass

class IsLoggedInPermissions (Permissions):
    def last_login():
        return auth.last_log